안녕하세요. XGBoost Regressor 와 간단한 feature selection 을 이용한 코드입니다. 상세 설명은 코드의 주석을 참고 바랍니다. 혹시 질문이나 잘못된 부분에 대해 덧글 부탁드립니다. 감사합니다.

# 1. 구현 환경

In [ ]:
!lscpu | egrep 'Model name|Socket|Thread|NUMA|CPU\(s\)'

In [ ]:
!lspci | grep VGA

# 2. 패키지

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1' # 사용하려는 GPU 번호
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import IPython.display as ipd
%matplotlib inline

# 3. 상수 설정

Standardizatino 과 min-max scaling 모두 적용해보았으나 아무것도 적용하지 않은 경우 점수가 더 높았습니다.

In [ ]:
SCALE = False

# 4. 데이터 불러오기

In [ ]:
trainset = pd.read_csv('./data2/train.csv')

In [ ]:
# 결측치 확인
np.sum(trainset.isna())

In [ ]:
testset = pd.read_csv('./data2/test.csv')

In [ ]:
# 결측치 확인
np.sum(testset.isna())

In [ ]:
# 최종 제출을 위한 dataframe 불러오기
submission = pd.read_csv('./data2/sample_submission.csv')

원본 dataframe 의 date_time 칼럼은 기계학습 알고리즘에 그대로 적용하기에 정보가 부족합니다. 그래서 정보를 상세하게 풀어서 추가해주었습니다. 
* week: 연도 별로 몇째 주 마다 이용자가 다를 수 있으므로 요일 정보를 추가해줍니다.
* weekday: 요일별로 따릉이 이용자가 다를 수 있으므로 요일 정보를 추가해줍니다.
* <b>추가 해볼 수 있는 것: 평일 / 주말 이용자가 다를 수 있으므로 평일과 주말에 대한 칼럼도 추가해주면 성능 향상에 도움이 될 듯 합니다.</b>

In [ ]:
def split_date(df):
    '''
    args
        df: 원본 데이터프레임
    return
        year, month, day, week, weekday 정보를 추가한 dataframe
    '''
    df['year'] = pd.to_datetime(df['date_time']).dt.year
    df['month'] = pd.to_datetime(df['date_time']).dt.month
    df['day'] = pd.to_datetime(df['date_time']).dt.day
    df['week'] = pd.to_datetime(df['date_time']).dt.week
    df['weekday'] = pd.to_datetime(df['date_time']).dt.weekday
    # df['quarter'] = pd.to_datetime(df['date_time']).dt.quarter # 데이터는 모두 동일 분기 데이터이므로 무의미
    
    return df.drop(columns=['date_time'])

In [ ]:
traindf = split_date(trainset)
testdf = split_date(testset)

In [ ]:
keys = traindf.drop(columns=['number_of_rentals']).keys()

In [ ]:
traindf.head()

In [ ]:
testdf.head()

In [ ]:
keydict = dict()
for i, k in enumerate(keys):
    keydict[i]=k

데이터 학습을 위해 입력값과 출력값을 분리해줍니다.

In [ ]:
X = np.array(traindf.drop(columns=['number_of_rentals'])).astype(float)
y = np.array(traindf['number_of_rentals']).astype(float)
X_test = np.array(testdf).astype(float)

if SCALE:
    x_max = np.max(X, 0)
    y_max = np.max(y, 0)
    X /= x_max
    y /= y_max
    X_test /= x_max

# 5. 유틸리티

In [ ]:
def nmae(true, pred):
    return np.mean(np.abs(true-pred)/true)

성능 확인을 위해서 몬테카를로 교차검증을 사용합니다([링크](https://en.wikipedia.org/wiki/Cross-validation_(statistics)의 섹션 Repeated random sub-sampling validation 참고).

In [ ]:
def cv(model, X=X, y=y, cnt=100, test_size=.5):
    score = list()
    seeds = np.random.choice(np.arange(1e5), size=cnt, replace=False)
    for seed in seeds:
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_valid)
        if SCALE:
            y_pred *= y_max
            y_valid *= y_max
        score.append(nmae(y_valid, y_pred))
    return np.mean(score)

# 6. Feature selection

제거 법을 이용해서 feature selection 을 해봅니다.

In [ ]:
def select_features(X, cols):
    _X = list()
    for c in cols:
        _X.append(np.expand_dims(X[:, c],-1))
    return np.concatenate(_X, axis=1)

In [ ]:
keydict

다양한 모델 확인 결과 XGBRegressor 가 가장 좋아서 이걸 선택했습니다.

In [ ]:
from xgboost import XGBRegressor as Model

In [ ]:
min_loss = 1e10
min_cols = None

for i in np.arange(start=0, stop=len(keys)):
    model = Model(tree_method='gpu_hist')
    cols = np.arange(len(keys))
    cols = np.delete(cols, i)
    _X = select_features(X, cols)
    loss = cv(model, _X, y, cnt=30)
    if min_loss > loss:
        min_loss = loss
        min_cols = f"{keydict.get(i)}"
    print(f"{keydict.get(i)}, {loss:.4f} (min {min_loss:.4f})")

In [ ]:
for i in np.arange(start=0, stop=len(keys)-1):
    for j in np.arange(start=i+1, stop=len(keys)):
        model = Model(tree_method='gpu_hist')
        cols = np.arange(len(keys))
        cols = np.delete(cols, [i,j])
        _X = select_features(X, cols)
        loss = cv(model, _X, y, cnt=30)
        if min_loss > loss:
            min_loss = loss
            min_cols = f"{keydict.get(i)}, {keydict.get(j)}"
        print(f"{keydict.get(i)}, {keydict.get(j)}, {loss:.4f} (min {min_loss:.4f})")

0: 'wind_direction' 과  7: 'Precipitation_Probability' 가 없을 때 성능이 좋을듯 합니다. 풍향은 자전거 이용에 영향을 미치는 변수가 아닐 듯 합니다. 한편, 강수확률(Precipitation_Probability)은 관측치가 아니라 기상청 예측치라면, 예측 오류가 우리가 만드는 최종 모델의 오류를 더 커지게 할 수 있기 때문에 빼는 것이 좋겠습니다.

# 7. 모델 학습

In [ ]:
model = Model(tree_method='gpu_hist')

cols = np.arange(len(keys))
cols = np.delete(cols, [0,7]) # 풍향, 강수확률 제거
_X = select_features(X, cols)

model.fit(_X, y)

_X_test = select_features(X_test, cols)
y_pred = model.predict(_X_test)
if SCALE:
    y_pred *= y_max
y_pred = y_pred.astype(int)

plt.scatter(np.arange(len(y_pred)), y_pred, s=.5)
plt.show()

In [ ]:
submission['number_of_rentals'] = y_pred

In [ ]:
submission.to_csv('./submission/submission.csv', index=False)

감사합니다.